In [2]:
# MIT License

# Copyright (c) 2023 Nathan Zorndorf

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.


import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
# environment 
# perform actions (action set based on current state), get next_state and reward 
# setup action array - 2D array with 

In [1]:
class GamblersProblem:
    def __init__(self, p_h) -> None:
        self.p_h = p_h
        self.state = 50 
        self.state_space = list(range(0,100))
        return
    
    def reset(self):
        self.state = 50 
        return
    
    def action_space(self):
        return list(range(0, min(self.state, 100-self.state))) # caps action space to [0-50] and thus resulting state space to [0,100]
    
    def random_choice(self):
        choices = [1, -1]
        probabilities = [self.p_h, 1 - self.p_h]
        result = np.random.choice(choices, p=probabilities)
        return result
    
    def step(self, state, action):
        """
        Run one timestep of the environment's dynamics.

        Args:
            state (int): An int storing the number of dollars available to bet
            action (int): The number of dollars to bet 

        Returns:
            (Tuple[int, float]): The new state of the system, and its correspondent reward
        """
        # if state == terminal state, return associated reward 
        if state == 0:
            reward = 0
            next_state = state 
        elif state == 100:
            reward = 1  
            next_state = state 
        else:
            reward = 0 
            next_state = action * self.random_choice() # sample from bernoulli distribution with probability p_h 

        return next_state, reward 
    
    def bellman_expectation(self, state, action):
        discount = 1
        next_state, _ = self.step(state, action)
        sum(
            self.p_h * (int(state + action >= 100) + discount * self.value_function[state + action]),
            (1-self.p_h) * (0 + discount * self.value_function[state - action])

        )
        return 
    
    def value_iteration(self, epsilon=0.01):
        
        # initialize value function 
        self.value_function = np.zeros(len(self.state_space))

        delta = 0 
        while(delta >= epsilon):   
            for state in self.state_space:
                old_v = self.value_function[state]
                self.value_function[state] = max(
                    [self.bellman_expectation(state, action) for action in self.action_space(state)]
                )

    


    

In [2]:
env = GamblersProblem(p_h=0.4)
env.step()